In [1]:
!pip install -q pypdf langchain langchain-community langchain-text-splitters
!pip install -q sentence_transformers faiss-gpu tiktoken sentence-transformers
!pip install sentencepiece
!pip install -q accelerate==0.27.2 peft==0.8.2 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!pip install 'transformers @ git+https://github.com/huggingface/transformers.git@831bc25d8fdb85768402f772cf65cc3d7872b211'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
! gdown 1IIOZcpDSm8DS8atRAqLun6Fyur9pHSHH

Downloading...
From: https://drive.google.com/uc?id=1IIOZcpDSm8DS8atRAqLun6Fyur9pHSHH
To: /content/data.pdf
100% 1.19M/1.19M [00:00<00:00, 107MB/s]


In [11]:
def rag(input_p):
  from langchain_community.document_loaders import PyPDFLoader
  book_path = '/content/data.pdf'
  loader = PyPDFLoader(book_path)
  pages = loader.load_and_split()

  # clean pages
  for page in pages:
      content = page.page_content
      len_before = len(content)
      content = content.replace("\n", "").replace("\t", "").replace("-", "")
      len_after = len(content)
      page.page_content = content


  from langchain_text_splitters import RecursiveCharacterTextSplitter
  splitter = RecursiveCharacterTextSplitter(
      chunk_size=10_000_000,
      chunk_overlap=50_000,
      length_function=len,
      is_separator_regex=False,
  )

  splitted_docs = splitter.transform_documents(pages)

  texts = []
  for doc in splitted_docs:
      texts.append(doc.page_content)

  from langchain_community.embeddings import HuggingFaceEmbeddings
  model_kwargs = {'device': 'cuda'}
  encode_kwargs = {'normalize_embeddings': False}
  embedding_model_id = "asafaya/bert-large-arabic"
  # embedding_model_id = "meta-llama/Meta-Llama-3-8B"
  # embedding_model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

  hf = HuggingFaceEmbeddings(
      model_name = embedding_model_id,
      model_kwargs=model_kwargs,
      encode_kwargs=encode_kwargs
  )

  from langchain_community.vectorstores import FAISS
  db = FAISS.from_documents(splitted_docs, hf)

  db.save_local("faiss_db")
  retriever = db.as_retriever()
  docs = retriever.get_relevant_documents(input_p)
  return docs[0].page_content

In [12]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    # HfArgumentParser,
    # TrainingArguments,
    pipeline,
    # logging,
)
from peft import LoraConfig, PeftModel

In [14]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"


################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [15]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
#tokenizer = AutoTokenizer.from_pretrained('mrm8488/phi-2-coder', use_fast=False)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
)

In [23]:
def generate_arabic_answer(question):

    # Generate response using the model
    generation = generator(question, max_length=1500, num_return_sequences=1)

    # Extract the generated text and handle potential issues with generation
    if generation:
        answer = generation[0]['generated_text']
        print("Question:", question)
        print("Answer:", answer)
        return answer
    else:
        print("No valid answer was generated.")
        return None


In [30]:
def l_lama3(arabic_question):
  doc = rag(arabic_question)
  promt = f"{arabic_question} إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة {doc[:1000]}"
  generate_arabic_answer(promt)

In [31]:
arabic_question ="ما هو تعريف الحضارة الرومانية؟"
l_lama3(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هو تعريف الحضارة الرومانية؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة وبذلك تعرض المصريون لألوان العذاب لاعتناقهم مذهبا مخالفا لمذهب الإمبراطورية.    كان المجتمع المصري في ذلك العهد يحيا في أسوأ أيامه حيث السلطة والإدارة والوظائف في يد الرومان واليونان وبعض اليهود من المتقربين إلى الرومان، والمصريون محرومون حتى من التمتع بخيرات بلادهم، لفرض الضرائب الباهظة المتعددة  عليهم وكذلك حرمانهم من الاشتراك في الجيش، مما جعلهم يعيشون كالغرباء عنها، وإذا وصل بعضهم إلى منصب دينى فإننا نجد الاضطهاد ينصب على هؤلاء القادة الدينيين، مما حمل فريق كبير منهم على الفرار بعقيدته إلى الأديرة والكهوف المنتشرة في أنحاء مصر واتخذوا عادة التنسك التي أخذوها عن اليهود، فازداد عدد الأديار تبعا لذلك وكثر عدد أعضائها الذين أعطوا لأنفسهم حق الإعفاء من الوظائف غير المأجورة وغيرها، وشذ بعض من هؤلاء فاحترفوا السطو والنهب.    وعلى هذا نقول أن المجتمع المصري في هذه الفترة كان على طبقتين:    1_ طبقة الرومان: وهم المنعمون المرفهون الذين يسكنون الحواضر ويتمتعون بخيرات البلاد.    2_ طبقة المصريين: الطبقة ال

In [32]:
arabic_question ="ما هي أشهر الملوك والفراعنة في الحضارة المصرية القديمة؟"
l_lama3(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هي أشهر الملوك والفراعنة في الحضارة المصرية القديمة؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة ظهرت الكتابة لأول مرة مع الملكية على العلامات والتسميات للمواد التي وجدت في المقابر الملكية. كان هذا في المقام الأول  هو مهنة الكُتاب، الذين عملوا من مؤسسة مفتاح الحياة  أو دار الحياة. وقد ضم الأخير مكاتب ومكتبات (سميت دار الكتب) ومعامل ومراصد فلكية[109] وكانت أشهر قطع الأدب المصري القديم مثل نصوص الأهرام ونصوص التوابيت مكتوبةبالكلاسيكية المصرية.    من الأدب المصري القديم ظهر الحكيم بتاح حتب الذي كتب وصايا في الأخلاق والتعامل، يعتبر كتابه في الحكمة (تعاليم بتاح  حتب) أول كتاب في تاريخ البشرية عن الأخلاق وحسن السلوك . عاصر بتاح حتب الملك جد كا رع (2414  2375 قبلالميلاد، من الأسرة الخامسة) وعمل وزيرا له .   استمرت الكتابة الهيروغليفية الكلاسيكية بكونها لغة وطريقة الكتابة حتى حوالي 1300 ق.م. وبالنسبة لقواعد اللغة فقدوصلت ذروتها خلال الدولة الوسطى. ومعها كثرت الكتابة على البردي باستخدام شكل أخر من اللغة المصرية والذي بدأ في  الدولة الحديثة وفيما تلى، وقد تمثل هذا الشكل الحديث ف

In [33]:
arabic_question ="وفاه توت عنخ امون ؟"
l_lama3(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: وفاه توت عنخ امون ؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة أتون، وهاجم سلطة وقوة المؤسسة الكهنوتية في ذلك الوقت43]  إلى مدينة أخناتون الجديدة (تلالعمارنة حالياً)، ولم يكترث للشؤون الخارجية وانكب  على استيعاب نفسه في دينة الجديد وأسلوبه الفني. بعد وفاته، تم التخلي  سريعاً عن عبادة  أتون، وقام كل  من توت عنخ أمون وخپر خپرو رع آي، وحورمحب بمسح جميع التفاصيل العائدة  لبدعة  إخناتون، أو كما تعرف باسم فترة تل العمارنة[44]    وعند تولي رمسيس الثاني العرش، والمعروف أيضاً برمسيس العظيم، وذلك في حوالي1279  ق.م، عمل على بناء المزيد  من المعابد، وإقامة المزيد من التماثيل والمسلات، بالإضافة لإنجابه أطفال أكثر من أي فرعون في التاريخ45]في معركة قادش ضد الحيثيين، والتي أسفرت وبعد قتالٍ عنيف دام لأكثر من 15 عاماً، عن أول وأقدم معاهدة سلام عرفهاالتاريخ، وكان ذلك في العام 1258 ق.م46]  جعل مصر وأرضهاأرضاً مغرية للغزو من القوى الأجنبية، خاصةً الليبيين وشعوب البحر .تمكن الجيش في بداية الأمر من صد هذه الغزوات  وردعها، ولكن مع زيادة وتكثيف الغزوات فقد مصر السيطرة على أراضي سوريا وفلسطين. وز

In [34]:
arabic_question ="أين دفنت حتشبسوت؟"
l_lama3(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: أين دفنت حتشبسوت؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة أتون، وهاجم سلطة وقوة المؤسسة الكهنوتية في ذلك الوقت43]  إلى مدينة أخناتون الجديدة (تلالعمارنة حالياً)، ولم يكترث للشؤون الخارجية وانكب  على استيعاب نفسه في دينة الجديد وأسلوبه الفني. بعد وفاته، تم التخلي  سريعاً عن عبادة  أتون، وقام كل  من توت عنخ أمون وخپر خپرو رع آي، وحورمحب بمسح جميع التفاصيل العائدة  لبدعة  إخناتون، أو كما تعرف باسم فترة تل العمارنة[44]    وعند تولي رمسيس الثاني العرش، والمعروف أيضاً برمسيس العظيم، وذلك في حوالي1279  ق.م، عمل على بناء المزيد  من المعابد، وإقامة المزيد من التماثيل والمسلات، بالإضافة لإنجابه أطفال أكثر من أي فرعون في التاريخ45]في معركة قادش ضد الحيثيين، والتي أسفرت وبعد قتالٍ عنيف دام لأكثر من 15 عاماً، عن أول وأقدم معاهدة سلام عرفهاالتاريخ، وكان ذلك في العام 1258 ق.م46]  جعل مصر وأرضهاأرضاً مغرية للغزو من القوى الأجنبية، خاصةً الليبيين وشعوب البحر .تمكن الجيش في بداية الأمر من صد هذه الغزوات  وردعها، ولكن مع زيادة وتكثيف الغزوات فقد مصر السيطرة على أراضي سوريا وفلسطين. وزاد

In [35]:
arabic_question ="ما سبب انتحار كليوباترا وانطونيوس؟"
l_lama3(arabic_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما سبب انتحار كليوباترا وانطونيوس؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة بسبب حبه لكليوبترا كما يزعم الجهال)    و لم يقبل اوكتافيانوس ان ينتقل العرش من كليوباترا إلى أحد ابنائها، فاختارت ان تنتحر على ان تدخل روما في موكب النصر، عليها الذل والعار مسلسلة، وفضلت ان تقتل نفسها قتلة مقدسة بحية الكوبرا وكان ذلك في أغسطس سنة 30 ق.مأيضاً لم يكن انتحارها نتيجة حبها  وعدم استطاعتها العيش بدون انطونيوس كما يزغم الاخرين.  وبذلك انتهى عصر الدولة اليونانية والبطالمة.  العصر الرومانى  اصبحت مصر مقاطعة رومانية في 30 ق.م. بعد أن هَزَمّ أوكتافيوس (الإمبراطور الروماني المستقبلي باسم أغسطسخصمه مارك أنطوني وأطاح بعرش الملكة الفرعونية كليوباترا وضم المملكة البطلمية إلى الإمبراطورية الرومانية. شملت المقاطعة معظم مناطق مصر الحديثة باستثناء شبه جزيرة سيناء، التي غزاها تراجان لاحقاً. كانت مقاطعة كريت وبرقة تحد  إيجبتوس غرباً ويهودا (العربية البترائية  لاحقاً) شرقاً .     أصبحت المقاطعة بمثابة منتج رئيسي للحبوب للإمبراطورية وامتلكت اقتصاداً حضرياً متطوراً  للغاية. كانت إيجبتوس إلى حد  كبير أغ

In [36]:
arabic_question ="ما هو الغرض من بناء هرم خوفو؟ "
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هو الغرض من بناء هرم خوفو؟  إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة أتون، وهاجم سلطة وقوة المؤسسة الكهنوتية في ذلك الوقت43]  إلى مدينة أخناتون الجديدة (تلالعمارنة حالياً)، ولم يكترث للشؤون الخارجية وانكب  على استيعاب نفسه في دينة الجديد وأسلوبه الفني. بعد وفاته، تم التخلي  سريعاً عن عبادة  أتون، وقام كل  من توت عنخ أمون وخپر خپرو رع آي، وحورمحب بمسح جميع التفاصيل العائدة  لبدعة  إخناتون، أو كما تعرف باسم فترة تل العمارنة[44]    وعند تولي رمسيس الثاني العرش، والمعروف أيضاً برمسيس العظيم، وذلك في حوالي1279  ق.م، عمل على بناء المزيد  من المعابد، وإقامة المزيد من التماثيل والمسلات، بالإضافة لإنجابه أطفال أكثر من أي فرعون في التاريخ45]في معركة قادش ضد الحيثيين، والتي أسفرت وبعد قتالٍ عنيف دام لأكثر من 15 عاماً، عن أول وأقدم معاهدة سلام عرفهاالتاريخ، وكان ذلك في العام 1258 ق.م46]  جعل مصر وأرضهاأرضاً مغرية للغزو من القوى الأجنبية، خاصةً الليبيين وشعوب البحر .تمكن الجيش في بداية الأمر من صد هذه الغزوات  وردعها، ولكن مع زيادة وتكثيف الغزوات فقد مصر السيطرة على أراضي سوريا 

In [37]:
arabic_question ="ما هو ارتفاع هرم منقرع؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هو ارتفاع هرم منقرع؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة ظهرت الكتابة لأول مرة مع الملكية على العلامات والتسميات للمواد التي وجدت في المقابر الملكية. كان هذا في المقام الأول  هو مهنة الكُتاب، الذين عملوا من مؤسسة مفتاح الحياة  أو دار الحياة. وقد ضم الأخير مكاتب ومكتبات (سميت دار الكتب) ومعامل ومراصد فلكية[109] وكانت أشهر قطع الأدب المصري القديم مثل نصوص الأهرام ونصوص التوابيت مكتوبةبالكلاسيكية المصرية.    من الأدب المصري القديم ظهر الحكيم بتاح حتب الذي كتب وصايا في الأخلاق والتعامل، يعتبر كتابه في الحكمة (تعاليم بتاح  حتب) أول كتاب في تاريخ البشرية عن الأخلاق وحسن السلوك . عاصر بتاح حتب الملك جد كا رع (2414  2375 قبلالميلاد، من الأسرة الخامسة) وعمل وزيرا له .   استمرت الكتابة الهيروغليفية الكلاسيكية بكونها لغة وطريقة الكتابة حتى حوالي 1300 ق.م. وبالنسبة لقواعد اللغة فقدوصلت ذروتها خلال الدولة الوسطى. ومعها كثرت الكتابة على البردي باستخدام شكل أخر من اللغة المصرية والذي بدأ في  الدولة الحديثة وفيما تلى، وقد تمثل هذا الشكل الحديث في وثائق راميسيد الإدارة، وقصائد 

In [38]:
arabic_question ="كيف أثر محمد علي باشا على التنوع الثقافي في مصر؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: كيف أثر محمد علي باشا على التنوع الثقافي في مصر؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة بحلول خريف عام1981 قامت الحكومة بحملة اعتقالات واسعة شملت المنظمات الإسلامية ومسؤولي الكنيسة القبطية والكتاب والصحفيين ومفكرين يساريين وليبراليين ووصل عدد المعتقلين في السجون المصرية إلى 1536  معتقلاً وذلك علىإثر حدوث بوادر فتن واضطرابات شعبية رافضة للصلح مع إسرائيل ولسياسات الدولة الاقتصادية .   في 6 أكتوبر من عام1981 (بعد31 يوم من إعلان قرارات الاعتقال)، اغتيل في عرض عسكري كان يقام بمناسبة ذكرى حرب أكتوبر، وقام بالاغتيال خالد الإسلامبولي وحسين عباس وعطا طايل وعبد الحميد عبد السلام التابعين لمنظمة الجهاد الإسلامي التي كانت تعارض بشدة اتفاقية السلام مع إسرائيل، حيث اطلقوا الرصاص على الرئيس السادات مما أدى إلى  إصابته برصاصة في رقبته ورصاصة في صدره ورصاصة في قلبه. وجاء اغتيال السادات بعد أشهر قليلة من حادثة مقتل المشير أحمد بدوي وبعض القيادات العسكرية في تحطم طائرة الهليكوبتر بشكل غامض جدًا، مما فتح باب الشكوك حول وجود مؤامرة13[]14]  نائب الرئيس السابق محمد حسني مبارك.    يرى مؤيدو

In [39]:
arabic_question ="كم عدد انواع الرهبانية؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: كم عدد انواع الرهبانية؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة وبذلك تعرض المصريون لألوان العذاب لاعتناقهم مذهبا مخالفا لمذهب الإمبراطورية.    كان المجتمع المصري في ذلك العهد يحيا في أسوأ أيامه حيث السلطة والإدارة والوظائف في يد الرومان واليونان وبعض اليهود من المتقربين إلى الرومان، والمصريون محرومون حتى من التمتع بخيرات بلادهم، لفرض الضرائب الباهظة المتعددة  عليهم وكذلك حرمانهم من الاشتراك في الجيش، مما جعلهم يعيشون كالغرباء عنها، وإذا وصل بعضهم إلى منصب دينى فإننا نجد الاضطهاد ينصب على هؤلاء القادة الدينيين، مما حمل فريق كبير منهم على الفرار بعقيدته إلى الأديرة والكهوف المنتشرة في أنحاء مصر واتخذوا عادة التنسك التي أخذوها عن اليهود، فازداد عدد الأديار تبعا لذلك وكثر عدد أعضائها الذين أعطوا لأنفسهم حق الإعفاء من الوظائف غير المأجورة وغيرها، وشذ بعض من هؤلاء فاحترفوا السطو والنهب.    وعلى هذا نقول أن المجتمع المصري في هذه الفترة كان على طبقتين:    1_ طبقة الرومان: وهم المنعمون المرفهون الذين يسكنون الحواضر ويتمتعون بخيرات البلاد.    2_ طبقة المصريين: الطبقة الكادحة ا

In [40]:
arabic_question ="في أي عام تولى محمد علي باشا الحكم في مصر؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: في أي عام تولى محمد علي باشا الحكم في مصر؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة بحلول خريف عام1981 قامت الحكومة بحملة اعتقالات واسعة شملت المنظمات الإسلامية ومسؤولي الكنيسة القبطية والكتاب والصحفيين ومفكرين يساريين وليبراليين ووصل عدد المعتقلين في السجون المصرية إلى 1536  معتقلاً وذلك علىإثر حدوث بوادر فتن واضطرابات شعبية رافضة للصلح مع إسرائيل ولسياسات الدولة الاقتصادية .   في 6 أكتوبر من عام1981 (بعد31 يوم من إعلان قرارات الاعتقال)، اغتيل في عرض عسكري كان يقام بمناسبة ذكرى حرب أكتوبر، وقام بالاغتيال خالد الإسلامبولي وحسين عباس وعطا طايل وعبد الحميد عبد السلام التابعين لمنظمة الجهاد الإسلامي التي كانت تعارض بشدة اتفاقية السلام مع إسرائيل، حيث اطلقوا الرصاص على الرئيس السادات مما أدى إلى  إصابته برصاصة في رقبته ورصاصة في صدره ورصاصة في قلبه. وجاء اغتيال السادات بعد أشهر قليلة من حادثة مقتل المشير أحمد بدوي وبعض القيادات العسكرية في تحطم طائرة الهليكوبتر بشكل غامض جدًا، مما فتح باب الشكوك حول وجود مؤامرة13[]14]  نائب الرئيس السابق محمد حسني مبارك.    يرى مؤيدو سياست

In [41]:
arabic_question ="ما هي خصائص الحضارة الرومانية؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هي خصائص الحضارة الرومانية؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة ظهرت الكتابة لأول مرة مع الملكية على العلامات والتسميات للمواد التي وجدت في المقابر الملكية. كان هذا في المقام الأول  هو مهنة الكُتاب، الذين عملوا من مؤسسة مفتاح الحياة  أو دار الحياة. وقد ضم الأخير مكاتب ومكتبات (سميت دار الكتب) ومعامل ومراصد فلكية[109] وكانت أشهر قطع الأدب المصري القديم مثل نصوص الأهرام ونصوص التوابيت مكتوبةبالكلاسيكية المصرية.    من الأدب المصري القديم ظهر الحكيم بتاح حتب الذي كتب وصايا في الأخلاق والتعامل، يعتبر كتابه في الحكمة (تعاليم بتاح  حتب) أول كتاب في تاريخ البشرية عن الأخلاق وحسن السلوك . عاصر بتاح حتب الملك جد كا رع (2414  2375 قبلالميلاد، من الأسرة الخامسة) وعمل وزيرا له .   استمرت الكتابة الهيروغليفية الكلاسيكية بكونها لغة وطريقة الكتابة حتى حوالي 1300 ق.م. وبالنسبة لقواعد اللغة فقدوصلت ذروتها خلال الدولة الوسطى. ومعها كثرت الكتابة على البردي باستخدام شكل أخر من اللغة المصرية والذي بدأ في  الدولة الحديثة وفيما تلى، وقد تمثل هذا الشكل الحديث في وثائق راميسيد الإدارة، 

In [42]:
arabic_question ="متى تولى بطليموس الثالث حكم مصر؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: متى تولى بطليموس الثالث حكم مصر؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة وكانت  نصيب مصر من أحد كبار القواد العسكريين الذي صاحب الإسكندر في غزواته وكان أسمه بطليموس بن لا جوس  وبدأ حكم أسرته وأطلق عليها اسم الحكم البطلمى ليبدأ في مصر حكم الاحتلال البطلمى اليونانى    عندما تولى بطليموس الخامس العرش لقب بـ أبيفانس (الإله الظاهر) وكان لا يزال طفلاً  في الخامسة من العمر، وقد قامت  انقسامات وصراعات داخلية في عهده أدت إلى حرب اهلية  دموية في شوارع الإسكندرية، واستغل الملك السلوقى في سوريا أنطيوخس الثالث هذه الفرصة  للتدخل والاستيلاء على مصر والقضاء على أسرة البطالمة وضمها إلى أملاكه، فاحتل فينيقيا  وزحف بجيشه على مصر، وعندئذ تدخلت روما بالرغم من أنها كانت خارجة لتوها من الحرب الهانيبالية التي أنهكتها،  فأمرت أنطيوخس أن على الرغبة الأخيرة التي أبداها والد جلالته(راجع جوستينوس كاتب من القرن الثاني الميلادي   Justinus, Roman History, Bk 34, S. 2)    وقد اضطر أنطيوخس أمام موقف روما المتشدد أن يكتفى بانتصاراته وضم البلاد التي أستولى عليها خارج أراضى مصر  واعترف بوضع مصر الخاص إل

In [43]:
arabic_question ="كيف ساهم محمد علي باشا في تعزيز التقدم العلمي والتكنولوجي في مصر؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: كيف ساهم محمد علي باشا في تعزيز التقدم العلمي والتكنولوجي في مصر؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة بحلول خريف عام1981 قامت الحكومة بحملة اعتقالات واسعة شملت المنظمات الإسلامية ومسؤولي الكنيسة القبطية والكتاب والصحفيين ومفكرين يساريين وليبراليين ووصل عدد المعتقلين في السجون المصرية إلى 1536  معتقلاً وذلك علىإثر حدوث بوادر فتن واضطرابات شعبية رافضة للصلح مع إسرائيل ولسياسات الدولة الاقتصادية .   في 6 أكتوبر من عام1981 (بعد31 يوم من إعلان قرارات الاعتقال)، اغتيل في عرض عسكري كان يقام بمناسبة ذكرى حرب أكتوبر، وقام بالاغتيال خالد الإسلامبولي وحسين عباس وعطا طايل وعبد الحميد عبد السلام التابعين لمنظمة الجهاد الإسلامي التي كانت تعارض بشدة اتفاقية السلام مع إسرائيل، حيث اطلقوا الرصاص على الرئيس السادات مما أدى إلى  إصابته برصاصة في رقبته ورصاصة في صدره ورصاصة في قلبه. وجاء اغتيال السادات بعد أشهر قليلة من حادثة مقتل المشير أحمد بدوي وبعض القيادات العسكرية في تحطم طائرة الهليكوبتر بشكل غامض جدًا، مما فتح باب الشكوك حول وجود مؤامرة13[]14]  نائب الرئيس السابق محمد حسني مب

In [44]:
arabic_question ="من قام ببناء مكتبة الإسكندرية؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: من قام ببناء مكتبة الإسكندرية؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة بحلول خريف عام1981 قامت الحكومة بحملة اعتقالات واسعة شملت المنظمات الإسلامية ومسؤولي الكنيسة القبطية والكتاب والصحفيين ومفكرين يساريين وليبراليين ووصل عدد المعتقلين في السجون المصرية إلى 1536  معتقلاً وذلك علىإثر حدوث بوادر فتن واضطرابات شعبية رافضة للصلح مع إسرائيل ولسياسات الدولة الاقتصادية .   في 6 أكتوبر من عام1981 (بعد31 يوم من إعلان قرارات الاعتقال)، اغتيل في عرض عسكري كان يقام بمناسبة ذكرى حرب أكتوبر، وقام بالاغتيال خالد الإسلامبولي وحسين عباس وعطا طايل وعبد الحميد عبد السلام التابعين لمنظمة الجهاد الإسلامي التي كانت تعارض بشدة اتفاقية السلام مع إسرائيل، حيث اطلقوا الرصاص على الرئيس السادات مما أدى إلى  إصابته برصاصة في رقبته ورصاصة في صدره ورصاصة في قلبه. وجاء اغتيال السادات بعد أشهر قليلة من حادثة مقتل المشير أحمد بدوي وبعض القيادات العسكرية في تحطم طائرة الهليكوبتر بشكل غامض جدًا، مما فتح باب الشكوك حول وجود مؤامرة13[]14]  نائب الرئيس السابق محمد حسني مبارك.    يرى مؤيدو سياسته أنه الرئيس

In [45]:
arabic_question ="متى جاء الفتح الاسلامى على مصر ؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: متى جاء الفتح الاسلامى على مصر ؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة وبذلك تعرض المصريون لألوان العذاب لاعتناقهم مذهبا مخالفا لمذهب الإمبراطورية.    كان المجتمع المصري في ذلك العهد يحيا في أسوأ أيامه حيث السلطة والإدارة والوظائف في يد الرومان واليونان وبعض اليهود من المتقربين إلى الرومان، والمصريون محرومون حتى من التمتع بخيرات بلادهم، لفرض الضرائب الباهظة المتعددة  عليهم وكذلك حرمانهم من الاشتراك في الجيش، مما جعلهم يعيشون كالغرباء عنها، وإذا وصل بعضهم إلى منصب دينى فإننا نجد الاضطهاد ينصب على هؤلاء القادة الدينيين، مما حمل فريق كبير منهم على الفرار بعقيدته إلى الأديرة والكهوف المنتشرة في أنحاء مصر واتخذوا عادة التنسك التي أخذوها عن اليهود، فازداد عدد الأديار تبعا لذلك وكثر عدد أعضائها الذين أعطوا لأنفسهم حق الإعفاء من الوظائف غير المأجورة وغيرها، وشذ بعض من هؤلاء فاحترفوا السطو والنهب.    وعلى هذا نقول أن المجتمع المصري في هذه الفترة كان على طبقتين:    1_ طبقة الرومان: وهم المنعمون المرفهون الذين يسكنون الحواضر ويتمتعون بخيرات البلاد.    2_ طبقة المصريين: الطبقة 

In [46]:
arabic_question ="من هم أشهر وأهم الفراعنة في الحضارة المصرية القديمة؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: من هم أشهر وأهم الفراعنة في الحضارة المصرية القديمة؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة ظهرت الكتابة لأول مرة مع الملكية على العلامات والتسميات للمواد التي وجدت في المقابر الملكية. كان هذا في المقام الأول  هو مهنة الكُتاب، الذين عملوا من مؤسسة مفتاح الحياة  أو دار الحياة. وقد ضم الأخير مكاتب ومكتبات (سميت دار الكتب) ومعامل ومراصد فلكية[109] وكانت أشهر قطع الأدب المصري القديم مثل نصوص الأهرام ونصوص التوابيت مكتوبةبالكلاسيكية المصرية.    من الأدب المصري القديم ظهر الحكيم بتاح حتب الذي كتب وصايا في الأخلاق والتعامل، يعتبر كتابه في الحكمة (تعاليم بتاح  حتب) أول كتاب في تاريخ البشرية عن الأخلاق وحسن السلوك . عاصر بتاح حتب الملك جد كا رع (2414  2375 قبلالميلاد، من الأسرة الخامسة) وعمل وزيرا له .   استمرت الكتابة الهيروغليفية الكلاسيكية بكونها لغة وطريقة الكتابة حتى حوالي 1300 ق.م. وبالنسبة لقواعد اللغة فقدوصلت ذروتها خلال الدولة الوسطى. ومعها كثرت الكتابة على البردي باستخدام شكل أخر من اللغة المصرية والذي بدأ في  الدولة الحديثة وفيما تلى، وقد تمثل هذا الشكل الحديث في و

In [47]:
arabic_question ="متى حكمت حتشبسوت مصر؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: متى حكمت حتشبسوت مصر؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة أتون، وهاجم سلطة وقوة المؤسسة الكهنوتية في ذلك الوقت43]  إلى مدينة أخناتون الجديدة (تلالعمارنة حالياً)، ولم يكترث للشؤون الخارجية وانكب  على استيعاب نفسه في دينة الجديد وأسلوبه الفني. بعد وفاته، تم التخلي  سريعاً عن عبادة  أتون، وقام كل  من توت عنخ أمون وخپر خپرو رع آي، وحورمحب بمسح جميع التفاصيل العائدة  لبدعة  إخناتون، أو كما تعرف باسم فترة تل العمارنة[44]    وعند تولي رمسيس الثاني العرش، والمعروف أيضاً برمسيس العظيم، وذلك في حوالي1279  ق.م، عمل على بناء المزيد  من المعابد، وإقامة المزيد من التماثيل والمسلات، بالإضافة لإنجابه أطفال أكثر من أي فرعون في التاريخ45]في معركة قادش ضد الحيثيين، والتي أسفرت وبعد قتالٍ عنيف دام لأكثر من 15 عاماً، عن أول وأقدم معاهدة سلام عرفهاالتاريخ، وكان ذلك في العام 1258 ق.م46]  جعل مصر وأرضهاأرضاً مغرية للغزو من القوى الأجنبية، خاصةً الليبيين وشعوب البحر .تمكن الجيش في بداية الأمر من صد هذه الغزوات  وردعها، ولكن مع زيادة وتكثيف الغزوات فقد مصر السيطرة على أراضي سوريا وفلسطين. 

In [48]:
arabic_question ="كم كان عدد الحجارة المستخدمة في بناء هرم خوفو؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: كم كان عدد الحجارة المستخدمة في بناء هرم خوفو؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة كانت تستورد من وسط أفريقيا وتمتلك من قبل النبلاء والملوك فقط. وقد لاحظ هيرودوت أن المصريين هو الشعب الوحيد في  ذلك الوقت الذي ربى الحيوانات داخل منازلهم77] الفترتين، فترة ما قبل الأسر والعصر المتأخر، انتشرت بشكلكبير وشعبي للغاية عبادة الآلهة في شكل وهيئة حيواناتهم، مثل القط باستت، وأبو منجل تحوت، وقد ربيت هذه الحيوانات في المزارع بأعداد كبيرة لغرض طقوس التضحية[80]    مصر هي منطقة غنية بأحجار البناء والديكور مثل خام النحاس والرصاص والذهب، والأحجار شبه الكريمة. أتاحت هذه  الموارد الطبيعية لقدماء المصريين بناء النصب التذكارية، ونحت التماثيل، وصناعة الأدوات وأنماط المجوهرات81]التحنيط، استخدم المحنطون الملح من واد النطرون لصنع المومياء، والجبس لصنع الجص82]صخرية تحمل معادن خام في بعض الأماكن، حيث وجدت في صحراء سيناء، والوديان البعيدة في الصحراء الشرقية، مما  تطلب بعثات استكشافية كبيرة الحجم تحت سيطرة الدولة لجلب والحصول على الموارد الموجودة هناك. كما كان هناك مناجم  ذهب كثيفة في النوبة، و

In [49]:
arabic_question ="كيف تعامل محمد علي باشا مع القضايا المتعلقة بالتخطيط العمراني في مصر؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: كيف تعامل محمد علي باشا مع القضايا المتعلقة بالتخطيط العمراني في مصر؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة بحلول خريف عام1981 قامت الحكومة بحملة اعتقالات واسعة شملت المنظمات الإسلامية ومسؤولي الكنيسة القبطية والكتاب والصحفيين ومفكرين يساريين وليبراليين ووصل عدد المعتقلين في السجون المصرية إلى 1536  معتقلاً وذلك علىإثر حدوث بوادر فتن واضطرابات شعبية رافضة للصلح مع إسرائيل ولسياسات الدولة الاقتصادية .   في 6 أكتوبر من عام1981 (بعد31 يوم من إعلان قرارات الاعتقال)، اغتيل في عرض عسكري كان يقام بمناسبة ذكرى حرب أكتوبر، وقام بالاغتيال خالد الإسلامبولي وحسين عباس وعطا طايل وعبد الحميد عبد السلام التابعين لمنظمة الجهاد الإسلامي التي كانت تعارض بشدة اتفاقية السلام مع إسرائيل، حيث اطلقوا الرصاص على الرئيس السادات مما أدى إلى  إصابته برصاصة في رقبته ورصاصة في صدره ورصاصة في قلبه. وجاء اغتيال السادات بعد أشهر قليلة من حادثة مقتل المشير أحمد بدوي وبعض القيادات العسكرية في تحطم طائرة الهليكوبتر بشكل غامض جدًا، مما فتح باب الشكوك حول وجود مؤامرة13[]14]  نائب الرئيس السابق محمد حسن

In [50]:
arabic_question ="ما هي الإنجازات الثقافية والفنية المرتبطة بعصر محمد علي باشا؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هي الإنجازات الثقافية والفنية المرتبطة بعصر محمد علي باشا؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة بحلول خريف عام1981 قامت الحكومة بحملة اعتقالات واسعة شملت المنظمات الإسلامية ومسؤولي الكنيسة القبطية والكتاب والصحفيين ومفكرين يساريين وليبراليين ووصل عدد المعتقلين في السجون المصرية إلى 1536  معتقلاً وذلك علىإثر حدوث بوادر فتن واضطرابات شعبية رافضة للصلح مع إسرائيل ولسياسات الدولة الاقتصادية .   في 6 أكتوبر من عام1981 (بعد31 يوم من إعلان قرارات الاعتقال)، اغتيل في عرض عسكري كان يقام بمناسبة ذكرى حرب أكتوبر، وقام بالاغتيال خالد الإسلامبولي وحسين عباس وعطا طايل وعبد الحميد عبد السلام التابعين لمنظمة الجهاد الإسلامي التي كانت تعارض بشدة اتفاقية السلام مع إسرائيل، حيث اطلقوا الرصاص على الرئيس السادات مما أدى إلى  إصابته برصاصة في رقبته ورصاصة في صدره ورصاصة في قلبه. وجاء اغتيال السادات بعد أشهر قليلة من حادثة مقتل المشير أحمد بدوي وبعض القيادات العسكرية في تحطم طائرة الهليكوبتر بشكل غامض جدًا، مما فتح باب الشكوك حول وجود مؤامرة13[]14]  نائب الرئيس السابق محمد حسني مبارك.

In [51]:
arabic_question ="ما هي الابتكارات العسكرية التي أدخلها محمد علي باشا؟"
l_lama3(arabic_question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: ما هي الابتكارات العسكرية التي أدخلها محمد علي باشا؟ إليك بعض المستندات  لمساعدتك في الإجابة الصحيحة بحلول خريف عام1981 قامت الحكومة بحملة اعتقالات واسعة شملت المنظمات الإسلامية ومسؤولي الكنيسة القبطية والكتاب والصحفيين ومفكرين يساريين وليبراليين ووصل عدد المعتقلين في السجون المصرية إلى 1536  معتقلاً وذلك علىإثر حدوث بوادر فتن واضطرابات شعبية رافضة للصلح مع إسرائيل ولسياسات الدولة الاقتصادية .   في 6 أكتوبر من عام1981 (بعد31 يوم من إعلان قرارات الاعتقال)، اغتيل في عرض عسكري كان يقام بمناسبة ذكرى حرب أكتوبر، وقام بالاغتيال خالد الإسلامبولي وحسين عباس وعطا طايل وعبد الحميد عبد السلام التابعين لمنظمة الجهاد الإسلامي التي كانت تعارض بشدة اتفاقية السلام مع إسرائيل، حيث اطلقوا الرصاص على الرئيس السادات مما أدى إلى  إصابته برصاصة في رقبته ورصاصة في صدره ورصاصة في قلبه. وجاء اغتيال السادات بعد أشهر قليلة من حادثة مقتل المشير أحمد بدوي وبعض القيادات العسكرية في تحطم طائرة الهليكوبتر بشكل غامض جدًا، مما فتح باب الشكوك حول وجود مؤامرة13[]14]  نائب الرئيس السابق محمد حسني مبارك.    يرى م